In [44]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/notebook


In [45]:
from langchain_community.document_loaders import (
                                                  PyMuPDFLoader,
                                                  NewsURLLoader,
                                                  YoutubeLoader,
                                                  )
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter, NLTKTextSplitter, SpacyTextSplitter
import pymupdf

import os
import torch
import re
from PIL import Image
from natsort import natsorted
import shutil
from transformers import pipeline
from src.utils import *
from src.data_processing import MathLatexRecovery

In [46]:
device = take_device()

Selected device: cuda


In [47]:
data = config_parser(data_config_path = 'config/data_config.yaml')

In [48]:
pdf_path = "data/web_data/Lecture 3.pdf"
loader = PyMuPDFLoader(pdf_path)
ori_text = loader.load()
ori_text, duplicate_count = remove_duplicate_documents(ori_text[:90])
print(len(ori_text))

90


In [49]:
ori_text[84:86]

[Document(page_content="85\nExample\nn (op3): σDNO=5 (EMPLOYEE)\nq CS1a = 2000\nq CS6a = xDNO + sDNO = 2 + 80 = 82\nn (op4): σDNO=5 AND SALARY>30000 AND SEX='F' (EMPLOYEE)\nq CS6a-DNO = 82\nq CS4-SALARY = xSALARY + (b/2) = 3 + 2000/2 = 1003\nq CS6a-SEX = xSEX + sSEX = 1 + 5000 = 5001\nq => chose DNO=5 first and check the other conditions\n85\n", metadata={'source': 'data/web_data/Lecture 3.pdf', 'file_path': 'data/web_data/Lecture 3.pdf', 'page': 84, 'total_pages': 92, 'format': 'PDF 1.7', 'title': 'Lecture 3.pptx', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Microsoft: Print To PDF', 'creationDate': "D:20240229170808+07'00'", 'modDate': "D:20240229170808+07'00'", 'trapped': ''}),
 Document(page_content='86\nUsing Selectivity and Cost Estimates in \nQuery Optimization (7)\nExamples of Cost Functions for JOIN\nn Join selectivity (js)\n     js = | (R \uf077\n\uf076\n \n \n \n \nC S) | / | R x  S | = | (R \uf077\n\uf076\n \n \n \n \nC S) | / (|R| * |S |)\n \n 

In [50]:
ori_text = remove_common_prefix_from_documents(ori_text)

In [51]:
ori_text[:5]

[Document(page_content='1\nDATA ENGINEERING\nLECTURE 3\n', metadata={'source': 'data/web_data/Lecture 3.pdf', 'file_path': 'data/web_data/Lecture 3.pdf', 'page': 0, 'total_pages': 92, 'format': 'PDF 1.7', 'title': 'Lecture 3.pptx', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Microsoft: Print To PDF', 'creationDate': "D:20240229170808+07'00'", 'modDate': "D:20240229170808+07'00'", 'trapped': ''}),
 Document(page_content='QUERY PROCESSING \nOVERVIEW\nn Query to DBMS\nq Pre-processing\nq Optimization --> execution plan\nq Data access\nn Indexing\nn Merge sort\n2\n', metadata={'source': 'data/web_data/Lecture 3.pdf', 'file_path': 'data/web_data/Lecture 3.pdf', 'page': 1, 'total_pages': 92, 'format': 'PDF 1.7', 'title': 'Lecture 3.pptx', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Microsoft: Print To PDF', 'creationDate': "D:20240229170808+07'00'", 'modDate': "D:20240229170808+07'00'", 'trapped': ''}),
 Document(page_content='DATA ST

In [52]:
text_list = [txt.page_content for txt in ori_text]

In [53]:
# Use a pipeline as a high-level helper

pipe = pipeline("text-classification", model="CrissWang/bert-math", torch_dtype=torch.float16, device = take_device())

Selected device: cuda


In [54]:
outputs = pipe(text_list, batch_size=8, truncation="only_first",)
print(outputs[:5])

[{'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}]


In [55]:
# for input, output in zip(text_list, outputs):
#     print(f"output: {output['label']} - Input: {input}")
#     print("---------------")

In [56]:
math_indices = [index for index, output in enumerate(outputs) if output['label'].lower() == "math"]

# Print the list of indices where the label is "math"
print(math_indices[:5])
print(f"There are {len(math_indices)} / {len(ori_text)} pages has math")

[10, 11, 15, 17, 20]
There are 22 / 90 pages has math


In [57]:
def save_math_doc_image( math_indices, pdf_path, tmp_path="data/tmp"):
    doc = pymupdf.open(pdf_path)  # open document
    os.makedirs(tmp_path, exist_ok=True)
    image_paths = []
    for page in doc:  # iterate through the pages
        if not (page.number in math_indices):
            continue
        pix = page.get_pixmap()  # render page to an image
        save_path = f"{tmp_path}/%i.png" % page.number
        pix.save(save_path)  # store image as a PNG
        image_paths.append(save_path)
    return image_paths

tmp_path="data/tmp"
image_list = save_math_doc_image(math_indices, pdf_path,tmp_path)

In [58]:
nougat_processor = pipeline("image-to-text", model="facebook/nougat-base",  device = take_device(), max_new_tokens=1024 )

Selected device: cuda


In [59]:
image_list[:5]

['data/tmp/10.png',
 'data/tmp/11.png',
 'data/tmp/15.png',
 'data/tmp/17.png',
 'data/tmp/20.png']

In [60]:
outputs = nougat_processor(image_list, batch_size=8,)

In [61]:
for i, index in enumerate(math_indices):
    for item in outputs[i]:
        item['index'] = index

In [63]:
outputs[80:90]

[]

In [64]:
for i, item in enumerate(outputs):
    idx = int(item[0]['index'])
    ori_text[idx].page_content = outputs[i][0]["generated_text"]

In [ ]:
ori_text = combine_short_doc(ori_text, 100)

In [65]:
ori_text[80:90]

[Document(page_content='81\nUsing Selectivity and Cost Estimates in Query \nOptimization (5) \nExamples of Cost Functions for SELECT (cont.)\nn\nS4. Using an ordering index to retrieve multiple records:\n     For the comparison condition on a key field with an ordering index, \nCS4= x + (b/2) \nn\nS5. Using a clustering index to retrieve multiple records for an \nequality condition:\n      CS5= x + ┌ (s/bfr) ┐ \nn\nS6. Using a secondary (B+-tree) index:\n    For an equality comparison, CS6a= x + s (option 1 & 2); \n                                             CS6a= x + s + 1 (option 3);\n    For a comparison condition such as >, <, >=, or <=, \n       CS6b= x + (bI1/2) + (r/2)\n', metadata={'source': 'data/web_data/Lecture 3.pdf', 'file_path': 'data/web_data/Lecture 3.pdf', 'page': 80, 'total_pages': 92, 'format': 'PDF 1.7', 'title': 'Lecture 3.pptx', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Microsoft: Print To PDF', 'creationDate': "D:20240229170808+07'

In [ ]:
remove_and_recreate_folder(tmp_path)


In [ ]:
post_processor = MathLatexRecovery()
outputs = post_processor.recover_math(ori_text)

Selected device: cuda
There are 1 / 6 pages has math


In [ ]:
outputs[30:39]

[]